In [2]:
# Install required dependencies
!pip install requests beautifulsoup4 pandas matplotlib seaborn tqdm

In [4]:
#!/usr/bin/env python3
# WebKit Security Analyzer for iOS/macOS
# A practical implementation for analyzing web security vulnerabilities specific to WebKit-based browsers

import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import json
import base64
import urllib.parse
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor
from IPython.display import HTML, display

class WebKitSecurityAnalyzer:
    """
    A comprehensive security analyzer for WebKit-specific vulnerabilities
    targeting iOS and macOS web browsers and applications.
    """

    def __init__(self):
        """Initialize the WebKit Security Analyzer with detection patterns"""

        # User agent string for Safari on macOS
        self.safari_macos_ua = "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Safari/605.1.15"

        # User agent string for Safari on iOS
        self.safari_ios_ua = "Mozilla/5.0 (iPhone; CPU iPhone OS 16_0 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.0 Mobile/15E148 Safari/604.1"

        # Initialize the detection patterns for WebKit-specific XSS vulnerabilities
        self.webkit_xss_patterns = [
            # DOM-based XSS patterns specific to WebKit's handling
            r"<input\s+[^>]*(?:onfocus|onblur|onchange)=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>",
            r"<(?:div|span)\s+[^>]*(?:onmouseover|onmouseout)=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>",

            # WebKit-specific event handlers that may be leveraged for XSS
            r"<[^>]+ontouchstart=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>",
            r"<[^>]+ontouchend=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>",
            r"<[^>]+gesturestart=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>",

            # Shadow DOM manipulation patterns
            r"<[^>]+attachShadow[^>]*>",

            # WebKit's innerHTML handling can be exploited in certain contexts
            r"(?:innerHTML|outerHTML)\s*=\s*(?:'|\")[^'\"]*<script[^>]*>[^<]*<\/script>(?:'|\")",

            # iOS/macOS specific JavaScript protocol handlers
            r"<[^>]+href=(?:'|\")(?:javascript|vbscript):(?:'|\")[^>]*>",

            # WebKit-specific CSS injection patterns
            r"<style[^>]*>@import\s+url\([\"']?(?:data|javascript):[^)]+\)[\"']?\s*;?<\/style>",

            # HTML5 elements with JavaScript execution capabilities
            r"<(?:video|audio)\s+[^>]*onerror=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>",
        ]

        # CSP bypass techniques specific to WebKit's implementation
        self.csp_bypass_patterns = [
            # Detecting CSP configurations vulnerable to Safari-specific bypasses
            r"script-src[^;]+'unsafe-inline'",
            r"script-src[^;]+'unsafe-eval'",
            r"script-src[^;]*data:",

            # Missing or incomplete frame-ancestors directive
            r"(?!frame-ancestors)",

            # WebKit-specific CSP bypasses using object-src
            r"(?!object-src 'none')",

            # Detection of base-uri directive absence (allows base tag injection)
            r"(?!base-uri 'none'|base-uri 'self')",
        ]

        # WebView configuration vulnerability patterns
        self.webview_vulnerability_patterns = [
            # iOS WKWebView security configuration issues
            r"\.javaScriptEnabled\s*=\s*true",
            r"\.allowsInlineMediaPlayback\s*=\s*true",
            r"\.allowsFileAccessFromFileURLs\s*=\s*true",
            r"\.allowUniversalAccessFromFileURLs\s*=\s*true",

            # JavaScript interface exposure patterns
            r"\.addScriptMessageHandler\(",
            r"\.userContentController\.add\(",

            # WebView loading untrusted content
            r"\.loadRequest\(\s*(?:URL|NSURLRequest)",
            r"\.loadHTMLString\(",
            r"\.loadFileURL\(",
        ]

        # Security header patterns to check
        self.security_headers = {
            "Content-Security-Policy": None,
            "X-Frame-Options": None,
            "X-Content-Type-Options": None,
            "Strict-Transport-Security": None,
            "X-XSS-Protection": None,
            "Referrer-Policy": None,
        }

        # Initialize results storage
        self.results_database = []

    def analyze_url(self, url, use_ios_ua=False):
        """
        Perform comprehensive security analysis of a URL with WebKit-specific focus

        Args:
            url (str): The URL to analyze
            use_ios_ua (bool): Whether to use iOS Safari user agent

        Returns:
            dict: Analysis results
        """
        results = {
            'url': url,
            'timestamp': pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S"),
            'status': 'Unknown',
            'headers': {},
            'security_headers': {},
            'xss_vulnerabilities': [],
            'csp_analysis': {},
            'webkit_specific_issues': [],
            'risk_score': 0.0,
            'mitigation_recommendations': []
        }

        # Select appropriate user agent
        user_agent = self.safari_ios_ua if use_ios_ua else self.safari_macos_ua

        try:
            # Make request with Safari user agent to simulate WebKit browser
            response = requests.get(
                url,
                headers={'User-Agent': user_agent},
                timeout=10,
                verify=True
            )

            results['status'] = response.status_code
            results['headers'] = dict(response.headers)

            # Extract security headers
            for header in self.security_headers:
                results['security_headers'][header] = response.headers.get(header)

            # Analyze Content Security Policy
            results['csp_analysis'] = self.analyze_csp(response.headers)

            # Parse response content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Scan for XSS vulnerabilities with WebKit-specific focus
            results['xss_vulnerabilities'] = self.scan_xss_patterns(response.text, soup)

            # Identify WebKit-specific security issues
            results['webkit_specific_issues'] = self.identify_webkit_issues(response.text, soup)

            # Calculate overall risk score based on findings
            results['risk_score'] = self.calculate_risk_score(results)

            # Generate mitigation recommendations
            results['mitigation_recommendations'] = self.generate_recommendations(results)

            # Add results to database
            self.results_database.append(results)

            return results

        except Exception as e:
            results['status'] = 'Error'
            results['error'] = str(e)
            return results

    def analyze_csp(self, headers):
        """
        Analyze Content Security Policy for WebKit-specific weaknesses

        Args:
            headers (dict): HTTP response headers

        Returns:
            dict: CSP analysis results
        """
        csp = headers.get('Content-Security-Policy', '')
        report_only_csp = headers.get('Content-Security-Policy-Report-Only', '')

        # Combine policies for analysis (if both exist)
        combined_csp = csp if not report_only_csp else f"{csp}; {report_only_csp}"

        results = {
            'has_csp': bool(csp),
            'has_report_only_csp': bool(report_only_csp),
            'policy': csp,
            'report_only_policy': report_only_csp,
            'directives': self.parse_csp_directives(combined_csp),
            'weaknesses': []
        }

        # Check for WebKit-specific CSP weaknesses
        if combined_csp:
            for pattern in self.csp_bypass_patterns:
                if re.search(pattern, combined_csp, re.IGNORECASE):
                    # Identify which specific bypass technique was found
                    weakness = self._identify_csp_weakness(pattern, combined_csp)
                    if weakness:
                        results['weaknesses'].append(weakness)
        else:
            # No CSP is a critical weakness
            results['weaknesses'].append({
                'type': 'missing_csp',
                'description': 'No Content Security Policy is defined',
                'severity': 'high',
                'webkit_specific': False
            })

        return results

    def _identify_csp_weakness(self, pattern, csp):
        """
        Identify specific CSP weakness based on matched pattern

        Args:
            pattern (str): The regex pattern that matched
            csp (str): The Content Security Policy string

        Returns:
            dict: Weakness details or None
        """
        # Map patterns to specific vulnerabilities
        if pattern == r"script-src[^;]+'unsafe-inline'":
            return {
                'type': 'unsafe_inline_script',
                'description': "CSP allows 'unsafe-inline' in script-src directive, enabling inline script execution",
                'severity': 'medium',
                'webkit_specific': False,
                'mitigation': "Remove 'unsafe-inline' from script-src and use nonces or hashes instead"
            }
        elif pattern == r"script-src[^;]+'unsafe-eval'":
            return {
                'type': 'unsafe_eval',
                'description': "CSP allows 'unsafe-eval' in script-src directive, enabling use of eval() and similar functions",
                'severity': 'medium',
                'webkit_specific': False,
                'mitigation': "Remove 'unsafe-eval' from script-src and refactor code to avoid dynamic code execution"
            }
        elif pattern == r"script-src[^;]*data:":
            return {
                'type': 'data_uri_script',
                'description': "CSP allows data: URIs in script-src directive, which can be used to bypass protections in WebKit browsers",
                'severity': 'high',
                'webkit_specific': True,
                'mitigation': "Remove data: URI scheme from script-src directive"
            }
        elif pattern == r"(?!frame-ancestors)":
            return {
                'type': 'missing_frame_ancestors',
                'description': "CSP is missing frame-ancestors directive, allowing potential clickjacking attacks",
                'severity': 'medium',
                'webkit_specific': False,
                'mitigation': "Add 'frame-ancestors 'none'' or 'frame-ancestors 'self'' to CSP"
            }
        elif pattern == r"(?!object-src 'none')":
            return {
                'type': 'missing_object_src',
                'description': "CSP is missing object-src directive, potentially allowing embedding of malicious objects",
                'severity': 'low',
                'webkit_specific': False,
                'mitigation': "Add 'object-src 'none'' to CSP"
            }
        elif pattern == r"(?!base-uri 'none'|base-uri 'self')":
            return {
                'type': 'missing_base_uri',
                'description': "CSP is missing base-uri directive, which can be exploited for relative path attacks in WebKit",
                'severity': 'medium',
                'webkit_specific': True,
                'mitigation': "Add 'base-uri 'self'' or 'base-uri 'none'' to CSP"
            }

        return None

    def parse_csp_directives(self, csp):
        """
        Parse CSP string into structured directives

        Args:
            csp (str): Content Security Policy string

        Returns:
            dict: Structured directives
        """
        if not csp:
            return {}

        directives = {}
        for directive in csp.split(';'):
            directive = directive.strip()
            if not directive:
                continue

            parts = directive.split()
            if parts:
                directive_name = parts[0]
                directive_values = parts[1:] if len(parts) > 1 else []
                directives[directive_name] = directive_values

        return directives

    def scan_xss_patterns(self, html_content, soup):
        """
        Scan for XSS vulnerabilities with focus on WebKit-specific patterns

        Args:
            html_content (str): Raw HTML content
            soup (BeautifulSoup): Parsed HTML content

        Returns:
            list: Detected vulnerabilities
        """
        vulnerabilities = []

        # Check for WebKit-specific XSS patterns
        for pattern in self.webkit_xss_patterns:
            matches = re.finditer(pattern, html_content, re.IGNORECASE)
            for match in matches:
                vulnerability = {
                    'type': 'webkit_xss',
                    'pattern': pattern,
                    'match': match.group(0),
                    'position': match.span(),
                    'context': html_content[max(0, match.start()-20):min(len(html_content), match.end()+20)],
                    'severity': self._determine_xss_severity(match.group(0)),
                    'description': self._describe_xss_pattern(pattern)
                }
                vulnerabilities.append(vulnerability)

        # Check for input fields without proper escaping
        input_fields = soup.find_all(['input', 'textarea'])
        for field in input_fields:
            # Check for reflection of user input without sanitization
            if field.get('value') and any(x in field.get('value', '') for x in ['<', '>', '"', "'"]):
                vulnerability = {
                    'type': 'input_reflection',
                    'element': str(field),
                    'severity': 'medium',
                    'description': 'User input is reflected in input field value without proper escaping',
                    'webkit_specific': True,
                    'reason': "WebKit's handling of reflected values in input fields has unique security implications"
                }
                vulnerabilities.append(vulnerability)

        # Check for event handlers in risky contexts
        elements_with_events = soup.find_all(lambda tag: any(attr.startswith('on') for attr in tag.attrs))
        for element in elements_with_events:
            for attr in element.attrs:
                if attr.startswith('on'):
                    # Check if event handler contains risky functionality
                    handler_value = element[attr]
                    if any(func in handler_value for func in ['eval', 'Function', 'setTimeout', 'setInterval']):
                        vulnerability = {
                            'type': 'webkit_event_handler',
                            'element': str(element),
                            'event': attr,
                            'handler': handler_value,
                            'severity': 'high',
                            'description': f'Risky event handler {attr} using eval or dynamic code execution',
                            'webkit_specific': True,
                            'reason': "WebKit's event handler execution model has specific security implications"
                        }
                        vulnerabilities.append(vulnerability)

        return vulnerabilities

    def _determine_xss_severity(self, match_content):
        """
        Determine the severity of an XSS vulnerability based on its content

        Args:
            match_content (str): The matched vulnerability content

        Returns:
            str: Severity level (high, medium, low)
        """
        # Higher severity for reflected input with direct script execution
        if 'eval(' in match_content or 'Function(' in match_content:
            return 'high'
        # Medium severity for event handlers that could execute code
        elif any(handler in match_content for handler in ['onclick', 'onload', 'onerror']):
            return 'medium'
        # Lower severity for potential issues that require specific conditions
        else:
            return 'low'

    def _describe_xss_pattern(self, pattern):
        """
        Generate a human-readable description of an XSS pattern

        Args:
            pattern (str): Regex pattern that was matched

        Returns:
            str: Human-readable description
        """
        # Map patterns to descriptions
        pattern_descriptions = {
            r"<input\s+[^>]*(?:onfocus|onblur|onchange)=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>":
                "Input field with event handler that executes JavaScript",

            r"<(?:div|span)\s+[^>]*(?:onmouseover|onmouseout)=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>":
                "Element with mouse event handler that executes JavaScript",

            r"<[^>]+ontouchstart=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>":
                "Element with touch event handler that executes JavaScript (iOS specific)",

            r"<[^>]+ontouchend=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>":
                "Element with touch event handler that executes JavaScript (iOS specific)",

            r"<[^>]+gesturestart=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>":
                "Element with gesture event handler that executes JavaScript (iOS specific)",

            r"<[^>]+attachShadow[^>]*>":
                "Element using Shadow DOM which can bypass certain protections in WebKit",

            r"(?:innerHTML|outerHTML)\s*=\s*(?:'|\")[^'\"]*<script[^>]*>[^<]*<\/script>(?:'|\")":
                "Dynamic HTML injection via innerHTML/outerHTML which WebKit may handle differently",

            r"<[^>]+href=(?:'|\")(?:javascript|vbscript):(?:'|\")[^>]*>":
                "Element with JavaScript protocol in href attribute",

            r"<style[^>]*>@import\s+url\([\"']?(?:data|javascript):[^)]+\)[\"']?\s*;?<\/style>":
                "CSS injection via @import with data or JavaScript URI scheme",

            r"<(?:video|audio)\s+[^>]*onerror=(?:'|\")[^'\"]*(?:eval|alert|confirm|prompt)(?:'|\")[^>]*>":
                "Media element with onerror event handler that executes JavaScript"
        }

        # Return description if found, otherwise provide a generic description
        return pattern_descriptions.get(pattern, "WebKit-specific cross-site scripting vulnerability")

    def identify_webkit_issues(self, html_content, soup):
        """
        Identify WebKit-specific security issues beyond XSS

        Args:
            html_content (str): Raw HTML content
            soup (BeautifulSoup): Parsed HTML content

        Returns:
            list: Detected security issues
        """
        issues = []

        # Check for WebKit-specific localStorage/sessionStorage usage
        storage_usage = re.findall(r'(?:localStorage|sessionStorage)\.setItem\([^)]+\)', html_content)
        if storage_usage:
            issues.append({
                'type': 'client_storage',
                'matches': storage_usage,
                'severity': 'low',
                'description': 'Usage of client-side storage which has unique handling in WebKit browsers',
                'webkit_specific': True,
                'reason': 'WebKit implements storage partitioning differently from other browsers'
            })

        # Check for vulnerable iframe usage
        iframes = soup.find_all('iframe')
        for iframe in iframes:
            if not iframe.get('sandbox') and iframe.get('src'):
                issues.append({
                    'type': 'vulnerable_iframe',
                    'element': str(iframe),
                    'severity': 'medium',
                    'description': 'Iframe without sandbox attribute can lead to security issues',
                    'webkit_specific': True,
                    'reason': 'WebKit has unique handling of iframe navigation that differs from other browsers'
                })

        # Check for META refresh redirections
        meta_refresh = soup.find_all('meta', attrs={'http-equiv': 'refresh'})
        for meta in meta_refresh:
            issues.append({
                'type': 'meta_refresh',
                'element': str(meta),
                'severity': 'low',
                'description': 'META refresh tag can be used for phishing in WebKit browsers',
                'webkit_specific': True,
                'reason': 'WebKit has specific behavior with META refresh that can be leveraged in attacks'
            })

        # Check for WebKit-specific CSS vulnerabilities
        style_tags = soup.find_all('style')
        for style in style_tags:
            if style.string and re.search(r'-webkit-', style.string):
                issues.append({
                    'type': 'webkit_css',
                    'element': str(style),
                    'severity': 'low',
                    'description': 'WebKit-specific CSS that may be used for browser fingerprinting',
                    'webkit_specific': True,
                    'reason': 'Vendor-specific CSS can be used to identify browser types which can aid in targeted attacks'
                })

        # Check for WebKit-specific viewport manipulation
        viewport_meta = soup.find('meta', attrs={'name': 'viewport'})
        if viewport_meta and 'user-scalable=no' in viewport_meta.get('content', ''):
            issues.append({
                'type': 'viewport_restriction',
                'element': str(viewport_meta),
                'severity': 'low',
                'description': 'Viewport prevents zooming which can aid in phishing attacks on iOS',
                'webkit_specific': True,
                'reason': 'This affects mobile Safari specifically and can be used in sophisticated phishing attacks'
            })

        # Check for usage of Web SQL (deprecated but still supported in WebKit)
        if re.search(r'openDatabase\(', html_content):
            issues.append({
                'type': 'web_sql',
                'severity': 'medium',
                'description': 'Usage of deprecated Web SQL which is only supported in WebKit browsers',
                'webkit_specific': True,
                'reason': 'Web SQL is deprecated but still supported in WebKit, creating potential for fingerprinting and bugs'
            })

        return issues

    def calculate_risk_score(self, results):
        """
        Calculate an overall risk score based on identified issues

        Args:
            results (dict): Analysis results

        Returns:
            float: Risk score between 0.0 (safe) and 10.0 (critical)
        """
        score = 0.0

        # Base score contributions
        if not results['security_headers'].get('Content-Security-Policy'):
            score += 2.0

        if not results['security_headers'].get('X-Frame-Options'):
            score += 1.0

        if not results['security_headers'].get('Strict-Transport-Security'):
            score += 1.0

        # XSS vulnerabilities significantly increase the score
        for vuln in results['xss_vulnerabilities']:
            if vuln['severity'] == 'high':
                score += 2.0
            elif vuln['severity'] == 'medium':
                score += 1.0
            else:
                score += 0.5

        # WebKit-specific issues add to the score
        for issue in results['webkit_specific_issues']:
            if issue['severity'] == 'high':
                score += 1.5
            elif issue['severity'] == 'medium':
                score += 0.75
            else:
                score += 0.25

        # CSP weaknesses
        for weakness in results['csp_analysis'].get('weaknesses', []):
            if weakness['severity'] == 'high':
                score += 1.5
            elif weakness['severity'] == 'medium':
                score += 1.0
            else:
                score += 0.5

        # Cap the score at 10
        return min(10.0, score)

    def generate_recommendations(self, results):
        """
        Generate security recommendations based on analysis results

        Args:
            results (dict): Analysis results

        Returns:
            list: Security recommendations
        """
        recommendations = []

        # CSP recommendations
        if not results['security_headers'].get('Content-Security-Policy'):
            recommendations.append({
                'type': 'csp',
                'priority': 'high',
                'description': 'Implement a Content Security Policy',
                'details': 'Add a Content Security Policy header to restrict script sources and prevent XSS attacks',
                'example': "Content-Security-Policy: default-src 'self'; script-src 'self'; object-src 'none'; frame-ancestors 'self';"
            })
        else:
            # Check for specific CSP improvements
            for weakness in results['csp_analysis'].get('weaknesses', []):
                if 'mitigation' in weakness:
                    recommendations.append({
                        'type': 'csp_improvement',
                        'priority': weakness['severity'],
                        'description': f"Improve CSP: {weakness['type']}",
                        'details': weakness['mitigation']
                    })

        # Security header recommendations
        if not results['security_headers'].get('X-Frame-Options'):
            recommendations.append({
                'type': 'header',
                'priority': 'medium',
                'description': 'Implement X-Frame-Options header',
                'details': 'Add X-Frame-Options header to prevent clickjacking attacks',
                'example': 'X-Frame-Options: DENY'
            })

        if not results['security_headers'].get('Strict-Transport-Security'):
            recommendations.append({
                'type': 'header',
                'priority': 'high',
                'description': 'Implement HTTP Strict Transport Security',
                'details': 'Add Strict-Transport-Security header to enforce HTTPS connections',
                'example': 'Strict-Transport-Security: max-age=31536000; includeSubDomains'
            })

        if not results['security_headers'].get('X-Content-Type-Options'):
            recommendations.append({
                'type': 'header',
                'priority': 'medium',
                'description': 'Implement X-Content-Type-Options header',
                'details': 'Add X-Content-Type-Options header to prevent MIME type sniffing',
                'example': 'X-Content-Type-Options: nosniff'
            })

        # WebKit-specific recommendations
        has_webkit_issues = any(issue.get('webkit_specific', False) for issue in results['webkit_specific_issues'])
        if has_webkit_issues:
            recommendations.append({
                'type': 'webkit',
                'priority': 'high',
                'description': 'Address WebKit-specific security issues',
                'details': 'Fix identified security issues that specifically affect WebKit-based browsers like Safari on iOS and macOS'
            })

        # XSS vulnerability recommendations
        if results['xss_vulnerabilities']:
            recommendations.append({
                'type': 'xss',
                'priority': 'high',
                'description': 'Fix Cross-Site Scripting vulnerabilities',
                'details': 'Address identified XSS vulnerabilities by properly escaping user input and implementing CSP'
            })

        return recommendations

    def batch_analyze(self, urls, use_ios_ua=False, max_workers=5):
        """
        Analyze multiple URLs in parallel

        Args:
            urls (list): List of URLs to analyze
            use_ios_ua (bool): Whether to use iOS Safari user agent
            max_workers (int): Maximum number of parallel workers

        Returns:
            pandas.DataFrame: Analysis results
        """
        results = []

        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {executor.submit(self.analyze_url, url, use_ios_ua): url for url in urls}
            for future in tqdm(futures, desc="Analyzing URLs"):
                results.append(future.result())

        # Convert results to DataFrame for easier analysis
        return pd.DataFrame(results)

    def visualize_results(self, results=None):
        """
        Visualize analysis results with charts and graphs

        Args:
            results (list/DataFrame, optional): Results to visualize, uses stored results if None

        Returns:
            None (displays visualizations)
        """
        if results is None:
            # Use stored results if none provided
            if not self.results_database:
                print("No results available for visualization.")
                return
            results_df = pd.DataFrame(self.results_database)
        elif isinstance(results, pd.DataFrame):
            results_df = results
        else:
            results_df = pd.DataFrame(results)

        # Set up the visualization style
        plt.style.use('ggplot')
        sns.set(style="whitegrid")

        # Create figure for visualizations
        fig, axes = plt.subplots(2, 2, figsize=(14, 12))

        # 1. Risk score distribution
        sns.histplot(results_df['risk_score'], bins=10, kde=True, ax=axes[0, 0])
        axes[0, 0].set_title('Distribution of Risk Scores')
        axes[0, 0].set_xlabel('Risk Score (0-10)')
        axes[0, 0].set_ylabel('Count')

        # 2. Security headers presence
        if 'security_headers' in results_df.columns:
            header_data = []
            for _, row in results_df.iterrows():
                for header, value in row['security_headers'].items():
                    header_data.append({
                        'header': header,
                        'present': bool(value)
                    })
            header_df = pd.DataFrame(header_data)
            header_counts = header_df.groupby('header')['present'].value_counts().unstack().fillna(0)
            header_counts.plot(kind='bar', stacked=True, ax=axes[0, 1])
            axes[0, 1].set_title('Security Headers Implementation')
            axes[0, 1].set_xlabel('Header')
            axes[0, 1].set_ylabel('axes')
            [0, 1].set_ylabel('Count')
            axes[0, 1].legend(['Absent', 'Present'])

        # 3. Vulnerability types
        vuln_types = []
        for _, row in results_df.iterrows():
            for vuln in row.get('xss_vulnerabilities', []):
                vuln_types.append(vuln.get('type', 'unknown'))
            for issue in row.get('webkit_specific_issues', []):
                vuln_types.append(issue.get('type', 'unknown'))

        if vuln_types:
            vuln_counts = pd.Series(vuln_types).value_counts()
            vuln_counts.plot(kind='barh', ax=axes[1, 0])
            axes[1, 0].set_title('Types of Vulnerabilities Detected')
            axes[1, 0].set_xlabel('Count')
            axes[1, 0].set_ylabel('Vulnerability Type')

        # 4. CSP weaknesses
        csp_weaknesses = []
        for _, row in results_df.iterrows():
            for weakness in row.get('csp_analysis', {}).get('weaknesses', []):
                csp_weaknesses.append(weakness.get('type', 'unknown'))

        if csp_weaknesses:
            csp_counts = pd.Series(csp_weaknesses).value_counts()
            csp_counts.plot(kind='pie', ax=axes[1, 1], autopct='%1.1f%%')
            axes[1, 1].set_title('CSP Weaknesses Distribution')
            axes[1, 1].set_ylabel('')

        plt.tight_layout()
        plt.show()

        # Additional visualization: Recommendation priority distribution
        recommendation_priorities = []
        for _, row in results_df.iterrows():
            for rec in row.get('mitigation_recommendations', []):
                recommendation_priorities.append(rec.get('priority', 'unknown'))

        if recommendation_priorities:
            plt.figure(figsize=(8, 6))
            sns.countplot(y=recommendation_priorities, order=['high', 'medium', 'low'])
            plt.title('Recommendation Priorities')
            plt.xlabel('Count')
            plt.ylabel('Priority Level')
            plt.show()

    def export_results(self, filename='webkit_security_analysis.json'):
        """
        Export analysis results to a JSON file

        Args:
            filename (str): Output filename

        Returns:
            bool: Success status
        """
        try:
            with open(filename, 'w') as f:
                json.dump(self.results_database, f, indent=2)
            print(f"Results exported to {filename}")
            return True
        except Exception as e:
            print(f"Error exporting results: {e}")
            return False

    def generate_report(self, results=None, output_format='html'):
        """
        Generate a comprehensive security report

        Args:
            results (dict/list, optional): Results to include in report
            output_format (str): Output format ('html' or 'markdown')

        Returns:
            str: Report content
        """
        if results is None:
            if not self.results_database:
                return "No results available for report generation."
            results = self.results_database

        if isinstance(results, dict):
            results = [results]

        if output_format == 'html':
            report = self._generate_html_report(results)
        else:
            report = self._generate_markdown_report(results)

        return report

    def _generate_html_report(self, results):
        """
        Generate HTML format security report

        Args:
            results (list): Analysis results

        Returns:
            str: HTML report
        """
        html = """<!DOCTYPE html>
        <html>
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>WebKit Security Analysis Report</title>
            <style>
                body { font-family: Arial, sans-serif; line-height: 1.6; margin: 0; padding: 20px; color: #333; }
                h1 { color: #2c3e50; border-bottom: 2px solid #3498db; padding-bottom: 10px; }
                h2 { color: #2980b9; margin-top: 30px; }
                h3 { color: #3498db; }
                .report-meta { background-color: #f8f9fa; padding: 15px; border-radius: 5px; margin-bottom: 20px; }
                .severity-high { color: #e74c3c; font-weight: bold; }
                .severity-medium { color: #f39c12; font-weight: bold; }
                .severity-low { color: #2ecc71; font-weight: bold; }
                .recommendation { background-color: #e8f4fc; padding: 10px; margin: 10px 0; border-left: 4px solid #3498db; }
                .vulnerability { background-color: #feecf0; padding: 10px; margin: 10px 0; border-left: 4px solid #e74c3c; }
                .webkit-issue { background-color: #fff5e6; padding: 10px; margin: 10px 0; border-left: 4px solid #f39c12; }
                table { border-collapse: collapse; width: 100%; margin: 20px 0; }
                th, td { text-align: left; padding: 12px; }
                th { background-color: #2c3e50; color: white; }
                tr:nth-child(even) { background-color: #f2f2f2; }
                .risk-meter { width: 100%; background-color: #e0e0e0; border-radius: 5px; }
                .risk-value { height: 24px; border-radius: 5px; text-align: center; color: white; font-weight: bold; }
                pre { background-color: #f8f9fa; padding: 10px; overflow-x: auto; border-radius: 5px; }
                .summary { display: flex; justify-content: space-between; margin-bottom: 30px; }
                .summary-box { flex: 1; margin: 0 10px; padding: 15px; border-radius: 5px; text-align: center; }
                .summary-box h3 { margin-top: 0; }
                .high-risk { background-color: #feecf0; }
                .medium-risk { background-color: #fff5e6; }
                .low-risk { background-color: #e8f4fc; }
            </style>
        </head>
        <body>
            <h1>WebKit Security Analysis Report</h1>
        """

        # Add report generation timestamp
        html += f"""
            <div class="report-meta">
                <p><strong>Report Generated:</strong> {pd.Timestamp.now().strftime("%Y-%m-%d %H:%M:%S")}</p>
                <p><strong>Total URLs Analyzed:</strong> {len(results)}</p>
            </div>
        """

        # Add executive summary
        high_risk = len([r for r in results if r.get('risk_score', 0) >= 7.0])
        medium_risk = len([r for r in results if 4.0 <= r.get('risk_score', 0) < 7.0])
        low_risk = len([r for r in results if r.get('risk_score', 0) < 4.0])

        html += f"""
            <h2>Executive Summary</h2>
            <div class="summary">
                <div class="summary-box high-risk">
                    <h3>High Risk</h3>
                    <p>{high_risk} URLs</p>
                </div>
                <div class="summary-box medium-risk">
                    <h3>Medium Risk</h3>
                    <p>{medium_risk} URLs</p>
                </div>
                <div class="summary-box low-risk">
                    <h3>Low Risk</h3>
                    <p>{low_risk} URLs</p>
                </div>
            </div>
        """

        # Add detailed results for each URL
        html += "<h2>Detailed Analysis</h2>"

        for i, result in enumerate(results):
            url = result.get('url', 'Unknown URL')
            risk_score = result.get('risk_score', 0)

            # Determine risk level color
            if risk_score >= 7.0:
                risk_color = "#e74c3c"  # High risk - red
            elif risk_score >= 4.0:
                risk_color = "#f39c12"  # Medium risk - orange
            else:
                risk_color = "#2ecc71"  # Low risk - green

            html += f"""
                <h3>URL {i+1}: {url}</h3>
                <p><strong>Risk Score:</strong></p>
                <div class="risk-meter">
                    <div class="risk-value" style="width: {risk_score*10}%; background-color: {risk_color};">
                        {risk_score:.1f}/10
                    </div>
                </div>

                <h4>Security Headers</h4>
                <table>
                    <tr>
                        <th>Header</th>
                        <th>Status</th>
                    </tr>
            """

            for header, value in result.get('security_headers', {}).items():
                status = "Present" if value else "Missing"
                color = "#2ecc71" if value else "#e74c3c"
                html += f"""
                    <tr>
                        <td>{header}</td>
                        <td style="color: {color};">{status}</td>
                    </tr>
                """

            html += "</table>"

            # Add vulnerabilities
            if result.get('xss_vulnerabilities'):
                html += "<h4>XSS Vulnerabilities</h4>"
                for vuln in result.get('xss_vulnerabilities', []):
                    severity_class = f"severity-{vuln.get('severity', 'medium')}"
                    html += f"""
                        <div class="vulnerability">
                            <p><strong>Type:</strong> {vuln.get('type', 'Unknown')}</p>
                            <p><strong>Severity:</strong> <span class="{severity_class}">{vuln.get('severity', 'Unknown').upper()}</span></p>
                            <p><strong>Description:</strong> {vuln.get('description', 'No description available')}</p>
                            <p><strong>Context:</strong></p>
                            <pre>{vuln.get('context', 'No context available')}</pre>
                        </div>
                    """

            # Add WebKit-specific issues
            if result.get('webkit_specific_issues'):
                html += "<h4>WebKit-Specific Issues</h4>"
                for issue in result.get('webkit_specific_issues', []):
                    severity_class = f"severity-{issue.get('severity', 'medium')}"
                    html += f"""
                        <div class="webkit-issue">
                            <p><strong>Type:</strong> {issue.get('type', 'Unknown')}</p>
                            <p><strong>Severity:</strong> <span class="{severity_class}">{issue.get('severity', 'Unknown').upper()}</span></p>
                            <p><strong>Description:</strong> {issue.get('description', 'No description available')}</p>
                            {'<p><strong>Element:</strong><pre>' + issue.get('element', '') + '</pre></p>' if 'element' in issue else ''}
                            <p><strong>Reason:</strong> {issue.get('reason', 'No explanation available')}</p>
                        </div>
                    """

            # Add CSP analysis
            if 'csp_analysis' in result:
                html += "<h4>Content Security Policy Analysis</h4>"
                csp = result['csp_analysis'].get('policy', 'No CSP implemented')
                has_csp = result['csp_analysis'].get('has_csp', False)

                if has_csp:
                    html += f"""
                        <p><strong>Content Security Policy:</strong></p>
                        <pre>{csp}</pre>
                    """
                else:
                    html += """
                        <p class="severity-high">No Content Security Policy implemented</p>
                    """

                if result['csp_analysis'].get('weaknesses'):
                    html += "<p><strong>CSP Weaknesses:</strong></p>"
                    for weakness in result['csp_analysis'].get('weaknesses', []):
                        severity_class = f"severity-{weakness.get('severity', 'medium')}"
                        html += f"""
                            <div class="vulnerability">
                                <p><strong>Type:</strong> {weakness.get('type', 'Unknown')}</p>
                                <p><strong>Severity:</strong> <span class="{severity_class}">{weakness.get('severity', 'Unknown').upper()}</span></p>
                                <p><strong>Description:</strong> {weakness.get('description', 'No description available')}</p>
                                {'<p><strong>Mitigation:</strong> ' + weakness.get('mitigation', 'No mitigation available') + '</p>' if 'mitigation' in weakness else ''}
                            </div>
                        """

            # Add recommendations
            if result.get('mitigation_recommendations'):
                html += "<h4>Security Recommendations</h4>"
                for rec in result.get('mitigation_recommendations', []):
                    priority_class = f"severity-{rec.get('priority', 'medium')}"
                    html += f"""
                        <div class="recommendation">
                            <p><strong>Recommendation:</strong> {rec.get('description', 'Unknown')}</p>
                            <p><strong>Priority:</strong> <span class="{priority_class}">{rec.get('priority', 'Unknown').upper()}</span></p>
                            <p><strong>Details:</strong> {rec.get('details', 'No details available')}</p>
                            {'<p><strong>Example:</strong><pre>' + rec.get('example', '') + '</pre></p>' if 'example' in rec else ''}
                        </div>
                    """

        html += """
        </body>
        </html>
        """

        return html

    def _generate_markdown_report(self, results):
        """
        Generate Markdown format security report

        Args:
            results (list): Analysis results

        Returns:
            str: Markdown report
        """
        md = "# WebKit Security Analysis Report\n\n"

        # Add report generation timestamp
        md += f"**Report Generated:** {pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')}\n"
        md += f"**Total URLs Analyzed:** {len(results)}\n\n"

        # Add executive summary
        high_risk = len([r for r in results if r.get('risk_score', 0) >= 7.0])
        medium_risk = len([r for r in results if 4.0 <= r.get('risk_score', 0) < 7.0])
        low_risk = len([r for r in results if r.get('risk_score', 0) < 4.0])

        md += "## Executive Summary\n\n"
        md += f"- **High Risk:** {high_risk} URLs\n"
        md += f"- **Medium Risk:** {medium_risk} URLs\n"
        md += f"- **Low Risk:** {low_risk} URLs\n\n"

        # Add detailed results for each URL
        md += "## Detailed Analysis\n\n"

        for i, result in enumerate(results):
            url = result.get('url', 'Unknown URL')
            risk_score = result.get('risk_score', 0)

            # Determine risk level text
            if risk_score >= 7.0:
                risk_level = "HIGH"
            elif risk_score >= 4.0:
                risk_level = "MEDIUM"
            else:
                risk_level = "LOW"

            md += f"### URL {i+1}: {url}\n\n"
            md += f"**Risk Score:** {risk_score:.1f}/10 ({risk_level})\n\n"

            md += "#### Security Headers\n\n"
            md += "| Header | Status |\n"
            md += "| ------ | ------ |\n"

            for header, value in result.get('security_headers', {}).items():
                status = "Present" if value else "Missing"
                md += f"| {header} | {status} |\n"

            md += "\n"

            # Add vulnerabilities
            if result.get('xss_vulnerabilities'):
                md += "#### XSS Vulnerabilities\n\n"
                for vuln in result.get('xss_vulnerabilities', []):
                    md += f"- **Type:** {vuln.get('type', 'Unknown')}\n"
                    md += f"  - **Severity:** {vuln.get('severity', 'Unknown').upper()}\n"
                    md += f"  - **Description:** {vuln.get('description', 'No description available')}\n"
                    md += f"  - **Context:** `{vuln.get('context', 'No context available')}`\n\n"

            # Add WebKit-specific issues
            if result.get('webkit_specific_issues'):
                md += "#### WebKit-Specific Issues\n\n"
                for issue in result.get('webkit_specific_issues', []):
                    md += f"- **Type:** {issue.get('type', 'Unknown')}\n"
                    md += f"  - **Severity:** {issue.get('severity', 'Unknown').upper()}\n"
                    md += f"  - **Description:** {issue.get('description', 'No description available')}\n"
                    if 'element' in issue:
                        md += f"  - **Element:** `{issue.get('element', '')}`\n"
                    md += f"  - **Reason:** {issue.get('reason', 'No explanation available')}\n\n"

            # Add CSP analysis
            if 'csp_analysis' in result:
                md += "#### Content Security Policy Analysis\n\n"
                csp = result['csp_analysis'].get('policy', 'No CSP implemented')
                has_csp = result['csp_analysis'].get('has_csp', False)

                if has_csp:
                    md += f"**Content Security Policy:**\n```\n{csp}\n```\n\n"
                else:
                    md += "**No Content Security Policy implemented**\n\n"

                if result['csp_analysis'].get('weaknesses'):
                    md += "**CSP Weaknesses:**\n\n"
                    for weakness in result['csp_analysis'].get('weaknesses', []):
                        md += f"- **Type:** {weakness.get('type', 'Unknown')}\n"
                        md += f"  - **Severity:** {weakness.get('severity', 'Unknown').upper()}\n"
                        md += f"  - **Description:** {weakness.get('description', 'No description available')}\n"
                        if 'mitigation' in weakness:
                            md += f"  - **Mitigation:** {weakness.get('mitigation', 'No mitigation available')}\n\n"

            # Add recommendations
            if result.get('mitigation_recommendations'):
                md += "#### Security Recommendations\n\n"
                for rec in result.get('mitigation_recommendations', []):
                    md += f"- **Recommendation:** {rec.get('description', 'Unknown')}\n"
                    md += f"  - **Priority:** {rec.get('priority', 'Unknown').upper()}\n"
                    md += f"  - **Details:** {rec.get('details', 'No details available')}\n"
                    if 'example' in rec:
                        md += f"  - **Example:** `{rec.get('example', '')}`\n\n"

            md += "---\n\n"

        return md
                                  # Google Colab Demo Functions

def run_webkit_security_analyzer_demo():
    """Demonstrate the WebKit Security Analyzer with a sample website"""
    print("Initializing WebKit Security Analyzer...")
    analyzer = WebKitSecurityAnalyzer()

    # Sample websites for analysis
    sample_urls = [
        "https://example.com",
        "https://html5sec.org",  # Website with deliberate security test cases
        "https://developer.apple.com"
    ]

    print(f"\nAnalyzing {len(sample_urls)} sample URLs...")
    results = []

    for url in sample_urls:
        print(f"\nAnalyzing: {url}")
        result = analyzer.analyze_url(url)
        results.append(result)

        # Print a summary of findings
        print(f"Risk Score: {result['risk_score']:.1f}/10")
        print(f"XSS Vulnerabilities: {len(result['xss_vulnerabilities'])}")
        print(f"WebKit-Specific Issues: {len(result['webkit_specific_issues'])}")
        print(f"CSP Weaknesses: {len(result['csp_analysis'].get('weaknesses', []))}")

    # Visualize the results
    print("\nGenerating visualizations...")
    analyzer.visualize_results(results)

    # Generate an HTML report
    print("\nGenerating HTML report...")
    report = analyzer.generate_report(results, 'html')

    # Display the report in Colab
    display(HTML(report))

    # Export results to JSON
    analyzer.export_results('webkit_security_analysis_results.json')

    return analyzer, results

def analyze_custom_url():
    """Analyze a custom URL entered by the user"""
    url = input("Enter a URL to analyze (include http:// or https://): ")

    analyzer = WebKitSecurityAnalyzer()
    print(f"Analyzing {url}...")

    # Analyze with both iOS and macOS user agents
    print("\nAnalyzing with macOS Safari user agent:")
    macos_result = analyzer.analyze_url(url, use_ios_ua=False)

    print("\nAnalyzing with iOS Safari user agent:")
    ios_result = analyzer.analyze_url(url, use_ios_ua=True)

    # Compare results
    print("\nComparison of results between macOS and iOS:")

    # Risk score comparison
    print(f"Risk Score (macOS): {macos_result['risk_score']:.1f}/10")
    print(f"Risk Score (iOS): {ios_result['risk_score']:.1f}/10")

    # Vulnerability count comparison
    print(f"XSS Vulnerabilities (macOS): {len(macos_result['xss_vulnerabilities'])}")
    print(f"XSS Vulnerabilities (iOS): {len(ios_result['xss_vulnerabilities'])}")

    print(f"WebKit Issues (macOS): {len(macos_result['webkit_specific_issues'])}")
    print(f"WebKit Issues (iOS): {len(ios_result['webkit_specific_issues'])}")

    # Generate reports
    print("\nGenerating comparison report...")
    report = """
    <h1>WebKit Security Analysis: iOS vs macOS Comparison</h1>
    <h2>URL: {}</h2>

    <h3>Risk Score Comparison</h3>
    <div style="display: flex; margin-bottom: 20px;">
        <div style="flex: 1; padding: 10px; background-color: #f5f5f5; border-radius: 5px; margin-right: 10px;">
            <h4>macOS Safari</h4>
            <div style="height: 30px; background-color: #ddd; border-radius: 5px;">
                <div style="height: 100%; width: {}%; background-color: {}; border-radius: 5px; text-align: center; line-height: 30px; color: white;">
                    {}/10
                </div>
            </div>
        </div>
        <div style="flex: 1; padding: 10px; background-color: #f5f5f5; border-radius: 5px;">
            <h4>iOS Safari</h4>
            <div style="height: 30px; background-color: #ddd; border-radius: 5px;">
                <div style="height: 100%; width: {}%; background-color: {}; border-radius: 5px; text-align: center; line-height: 30px; color: white;">
                    {}/10
                </div>
            </div>
        </div>
    </div>
    """.format(
        url,
        macos_result['risk_score'] * 10,
        "#e74c3c" if macos_result['risk_score'] >= 7 else "#f39c12" if macos_result['risk_score'] >= 4 else "#2ecc71",
        round(macos_result['risk_score'], 1),
        ios_result['risk_score'] * 10,
        "#e74c3c" if ios_result['risk_score'] >= 7 else "#f39c12" if ios_result['risk_score'] >= 4 else "#2ecc71",
        round(ios_result['risk_score'], 1)
    )

    # Display the report
    display(HTML(report))

    # Generate detailed reports for both
    print("\nGenerating detailed reports...")
    macos_detailed = analyzer.generate_report(macos_result, 'html')
    ios_detailed = analyzer.generate_report(ios_result, 'html')

    # Offer to display detailed reports
    choice = input("\nWould you like to see detailed reports? (yes/no): ")
    if choice.lower() in ['yes', 'y']:
        print("\nDisplaying macOS detailed report:")
        display(HTML(macos_detailed))

        print("\nDisplaying iOS detailed report:")
        display(HTML(ios_detailed))

    return analyzer, macos_result, ios_result

def analyze_webview_code():
    """Analyze WebView code for potential security issues"""
    print("WebView Security Analyzer")
    print("-------------------------")
    print("This tool analyzes iOS/macOS WebView code for security vulnerabilities.")

    # Options for analysis
    print("\nSelect input method:")
    print("1. Enter code snippet")
    print("2. Load from file")
    choice = input("Enter choice (1 or 2): ")

    code = ""
    if choice == "1":
        print("\nEnter or paste WebView code (type 'END' on a new line when finished):")
        while True:
            line = input()
            if line == "END":
                break
            code += line + "\n"
    elif choice == "2":
        from google.colab import files
        print("\nUpload your WebView code file:")
        uploaded = files.upload()
        for filename in uploaded.keys():
            code = uploaded[filename].decode('utf-8')
    else:
        print("Invalid choice.")
        return None

    # Initialize the analyzer
    analyzer = WebKitSecurityAnalyzer()

    # Analyze the WebView code
    issues = []
    for pattern in analyzer.webview_vulnerability_patterns:
        matches = re.finditer(pattern, code, re.IGNORECASE)
        for match in matches:
            issue = {
                'pattern': pattern,
                'match': match.group(0),
                'position': match.span(),
                'context': code[max(0, match.start()-20):min(len(code), match.end()+20)],
                'description': _describe_webview_issue(pattern)
            }
            issues.append(issue)

    # Display results
    if issues:
        print(f"\nFound {len(issues)} potential security issues in WebView code:")
        for i, issue in enumerate(issues):
            print(f"\nIssue {i+1}:")
            print(f"  Description: {issue['description']}")
            print(f"  Matched code: {issue['match']}")
            print(f"  Context: {issue['context']}")
    else:
        print("\nNo security issues found in the provided WebView code.")

    # Generate recommendations
    if issues:
        print("\nRecommendations:")
        recommendations = _generate_webview_recommendations(issues)
        for i, rec in enumerate(recommendations):
            print(f"{i+1}. {rec}")

    return issues

def _describe_webview_issue(pattern):
    """Generate a description for a WebView vulnerability pattern"""
    descriptions = {
        r"\.javaScriptEnabled\s*=\s*true":
            "JavaScript is explicitly enabled, which could allow execution of malicious scripts if loading untrusted content",

        r"\.allowsInlineMediaPlayback\s*=\s*true":
            "Inline media playback is enabled, which could potentially be exploited for phishing or UI redressing attacks",

        r"\.allowsFileAccessFromFileURLs\s*=\s*true":
            "File access from file URLs is enabled, allowing loaded file:// content to access other local files",

        r"\.allowUniversalAccessFromFileURLs\s*=\s*true":
            "Universal access from file URLs is enabled, creating a significant security risk by allowing access to any resource",

        r"\.addScriptMessageHandler\(":
            "JavaScript interface is being exposed to WebView content, which could lead to privilege escalation if untrusted content is loaded",

        r"\.userContentController\.add\(":
            "JavaScript bridge is being created, exposing native functionality to web content",

        r"\.loadRequest\(\s*(?:URL|NSURLRequest)":
            "WebView is loading content from a URL - ensure the source is trusted and validate user input if the URL is dynamic",

        r"\.loadHTMLString\(":
            "WebView is loading HTML string content - ensure content is properly validated and sanitized if it contains user input",

        r"\.loadFileURL\(":
            "WebView is loading content from a file URL, which requires careful handling of security boundaries"
    }

    return descriptions.get(pattern, "Potential security issue in WebView configuration")

def _generate_webview_recommendations(issues):
    """Generate recommendations based on identified WebView issues"""
    recommendations = set


    for issue in issues:
        pattern = issue['pattern']

        if pattern == r"\.javaScriptEnabled\s*=\s*true":
            recommendations.add("Disable JavaScript when not necessary, especially when loading untrusted content.")

        if pattern == r"\.allowsFileAccessFromFileURLs\s*=\s*true":
            recommendations.add("Set allowsFileAccessFromFileURLs to false to prevent loaded file:// content from accessing other local files.")

        if pattern == r"\.allowUniversalAccessFromFileURLs\s*=\s*true":
            recommendations.add("Set allowUniversalAccessFromFileURLs to false to prevent file:// content from accessing any resource.")

        if pattern in [r"\.addScriptMessageHandler\(", r"\.userContentController\.add\("]:
            recommendations.add("Implement strict message validation in JavaScript interfaces to prevent privilege escalation.")
            recommendations.add("Limit exposed JavaScript interfaces to only what is necessary for functionality.")

        if pattern in [r"\.loadRequest\(\s*(?:URL|NSURLRequest)", r"\.loadHTMLString\(", r"\.loadFileURL\("]:
            recommendations.add("Validate and sanitize all content before loading into WebView, particularly user-provided content.")
            recommendations.add("Implement Content Security Policy (CSP) for loaded content to restrict capabilities.")

    # Add general recommendations
    recommendations.add("Implement WKNavigationDelegate to validate all navigation requests before allowing them.")
    recommendations.add("Avoid loading untrusted or user-provided content in WebViews with elevated privileges.")
    recommendations.add("Consider using SFSafariViewController for loading untrusted web content instead of WebView.")

    return list(recommendations)

# Main execution in Google Colab
if __name__ == "__main__":
    print("WebKit Security Analyzer for iOS/macOS")
    print("=====================================")
    print("This tool analyzes web security vulnerabilities specific to WebKit-based browsers on iOS and macOS.")

    print("\nSelect an option:")
    print("1. Run demo with sample websites")
    print("2. Analyze a custom URL")
    print("3. Analyze WebView code for security issues")

    choice = input("Enter your choice (1-3): ")

    if choice == "1":
        analyzer, results = run_webkit_security_analyzer_demo()
    elif choice == "2":
        analyzer, macos_result, ios_result = analyze_custom_url()
    elif choice == "3":
        issues = analyze_webview_code()
    else:
        print("Invalid choice. Please run again and select a valid option.")

WebKit Security Analyzer for iOS/macOS
This tool analyzes web security vulnerabilities specific to WebKit-based browsers on iOS and macOS.

Select an option:
1. Run demo with sample websites
2. Analyze a custom URL
3. Analyze WebView code for security issues
Enter your choice (1-3): 2
Enter a URL to analyze (include http:// or https://): https://appleid.apple.com
Analyzing https://appleid.apple.com...

Analyzing with macOS Safari user agent:

Analyzing with iOS Safari user agent:

Comparison of results between macOS and iOS:
Risk Score (macOS): 2.8/10
Risk Score (iOS): 2.8/10
XSS Vulnerabilities (macOS): 0
XSS Vulnerabilities (iOS): 0
WebKit Issues (macOS): 1
WebKit Issues (iOS): 1

Generating comparison report...



Generating detailed reports...

Would you like to see detailed reports? (yes/no): yes

Displaying macOS detailed report:


Header,Status
Content-Security-Policy,Present
X-Frame-Options,Present
X-Content-Type-Options,Present
Strict-Transport-Security,Present
X-XSS-Protection,Present
Referrer-Policy,Present



Displaying iOS detailed report:


Header,Status
Content-Security-Policy,Present
X-Frame-Options,Present
X-Content-Type-Options,Present
Strict-Transport-Security,Present
X-XSS-Protection,Present
Referrer-Policy,Present
